“It seems that the neural link protecting my core is not functioning correctly. The malfunctioning link has caused a flare-up in the pain receptors of my brain’s parietal lobe, and now, I have to stop the scientist from figuring out I am not human.”

Protect the humanoid from exposure by getting the protection of her core back online.

For now, you only have input data from several working fractions of the neural link connection. You know where the fractions work, and where they start malfunctioning. To repair the neural link connections, you need to figure out a working path through the core and generate a shield around it.

In the available input data, each line describes the route of a single neural strand – a fraction of a complete working neural link's path.

At the start of the line you’ll find the X and Y coordinates in a two-dimensional space, describing the starting point of the neural link's fraction. The rest of the line describes the path of the neural strand: D for down (+1 in the Y axis), U for up, R for right (+1 in the X axis), and L for left.

If a neural strand hits an object at the end of its path, it is shown as the last item: X for a wall, F for the finishing point, and S for the start point.

You need to provide a complete path beginning from the start point (S) and ending at any point in the finishing area (F) so that the path does not hit any walls along the way. You know that the neural paths’ routes before the malfunction are safe, up to the point where their route runs into a wall. Additionally, you know that neural strands are capable of forming a working connection as long as they are next to or touching each other.

Any two strands are considered connected if their paths cross, OR if they have one or more adjacent coordinates (left, right, down or up, but not diagonal).

Find a path for a working, complete neural connection. Use the directional instructions D, U, R and L as described above.

You should provide your answer as a string, containing characters L, R, U and D.

In [1]:
#!fsharp
#r "nuget: FParsec"

open FParsec


type Coordinate = { x: int ; y: int }

type PathInstruction =
    | Down
    | Up
    | Right
    | Left
    | Wall
    | Start
    | Finish

type PathInstructions = { startingCoordinate: Coordinate ; path : PathInstruction list }

let input = System.IO.File.ReadAllLines "./android_input.txt"

let parseCoordinate : Parser<Coordinate, unit> =
    pipe2
        (pint32 .>> pchar ',')
        (pint32)
        (fun a b -> { x = a ; y = b})

let parsePathElement : Parser<PathInstruction, unit> =
    choice 
        [ pchar 'D' >>. preturn Down
        ; pchar 'U' >>. preturn Up
        ; pchar 'R' >>. preturn Right
        ; pchar 'L' >>. preturn Left
        ; pchar 'X' >>. preturn Wall
        ; pchar 'S' >>. preturn Start
        ; pchar 'F' >>. preturn Finish
        ]

let parsePath : Parser<PathInstructions, unit> =
    pipe2
        (parseCoordinate .>> pchar ' ')
        (sepBy parsePathElement (pchar ','))
        (fun a b -> {startingCoordinate = a ; path = b} )

let pathInstructions = Array.map (run parsePath) input |> Array.choose (function | Success (r,_,_) -> Some r | Failure _ -> None) |> Array.toList

type PathElement =
    | Open of Coordinate
    | PathStart of Coordinate
    | PathFinish of Coordinate
    | PathWall of Coordinate

let coordinateOf (p: PathElement) =
    match p with
    | Open c -> c
    | PathStart c -> c
    | PathFinish c -> c
    | PathWall c -> c

type Path = PathElement array

type PathNumber = int 

type Index = int

type PathIndex = Map<Coordinate, (PathNumber * Index) list>

let pathFromInstructions (p: PathInstructions) =
    let rec _pathFromInstructions (acc: PathElement list) (currentCoordinate: Coordinate) (pathElements: PathInstruction list) =
        match pathElements with
        | Down :: rest -> _pathFromInstructions (Open currentCoordinate :: acc) {x = currentCoordinate.x ; y = currentCoordinate.y + 1} rest
        | Up :: rest -> _pathFromInstructions (Open currentCoordinate :: acc) {x = currentCoordinate.x ; y = currentCoordinate.y - 1} rest
        | Right :: rest -> _pathFromInstructions (Open currentCoordinate :: acc) {x = currentCoordinate.x + 1 ; y = currentCoordinate.y} rest
        | Left :: rest -> _pathFromInstructions (Open currentCoordinate :: acc) {x = currentCoordinate.x - 1 ; y = currentCoordinate.y} rest
        | Wall :: rest -> acc |> List.rev
        | Start :: rest -> (PathStart currentCoordinate) :: acc |> List.rev
        | Finish :: rest -> (PathFinish currentCoordinate) :: acc |> List.rev
        | [] -> List.rev acc

    _pathFromInstructions [] p.startingCoordinate p.path |> Array.ofList

let paths : Path array = List.map pathFromInstructions pathInstructions |> Array.ofList

let testPath = Array.head paths

let indexPath (pathNumber: PathNumber) (path: Path) =
    Array.mapi (fun ix p -> (coordinateOf p), (pathNumber, ix)) path

let pathIndex : PathIndex = 
    Array.mapi indexPath paths
    |> Array.concat
    |> Array.fold (fun state (k, v) ->
        Map.change k (function
        | Some a -> Some (v :: a)
        | None -> Some [v]
        ) state
    ) Map.empty

let getConnections (pathNumber: PathNumber) (ix: Index) (paths: Path array) (pathIndex: PathIndex) =
    let c = paths.[pathNumber].[ix] |> coordinateOf
    in
    [ { x = c.x - 1 ; y = c.y }; { x = c.x + 1 ; y = c.y }; { x = c.x; y = c.y - 1 }; { x = c.x ; y = c.y + 1 } ] 
    |> List.filter (fun c -> Map.containsKey c pathIndex)
    |> List.map (fun c -> Map.find c pathIndex)
    |> List.concat
    |> List.distinct
    |> List.filter (fun (num, _) -> num <> pathNumber)

let getNextOnPath (pathNumber: PathNumber) (index: Index) (paths: Path array) =
    [pathNumber, index + 1; pathNumber, index - 1] |> List.filter (fun (num, ix) -> 0 < ix && ix < Array.length paths.[pathNumber])


// starting coordinate is x = 2, y = 2

pathIndex.[{x = 2; y = 2}]



index,Item1,Item2
0,94,1
1,44,33
2,8,111


In [1]:
#!fsharp
let findPath (startCoordinate: Coordinate) (paths: Path array) (pathIndex: PathIndex) =
    let mutable distance : Map<PathNumber * Index, int> = Map.ofSeq (Map.toArray pathIndex |> Seq.map snd |> Seq.concat |> Seq.map (fun x -> x, 9999999))
    let mutable previous : Map<PathNumber * Index, PathNumber * Index> = Map.empty
    let mutable queue : Set<PathNumber * Index> = Set.ofArray (Array.mapi (fun ix x -> Array.mapi (fun ix2 _ -> ix, ix2) x) paths |> Array.concat)

    let startPoints = pathIndex.[startCoordinate]

    for s in startPoints do
        distance <- Map.add s 0 distance

    while (not (Set.isEmpty queue)) do
        let u = Seq.minBy (fun x -> distance.[x]) queue
        let u_pathNum, u_ix = u
        queue <- Set.remove u queue

        for v in (getNextOnPath u_pathNum u_ix paths @ getConnections u_pathNum u_ix paths pathIndex) do 
            let alt = distance.[u] + 1
            if alt < distance.[v] then
                distance <- Map.add v alt distance
                previous <- Map.add v u previous

    // Find the finish points
    let goals = 
        Array.mapi (fun ix x -> (Array.mapi (fun ix2 x2 -> (ix, ix2), x2) x)) paths 
        |> Array.concat 
        |> Array.filter (function | _, PathFinish _ -> true | _ -> false) 
        |> Array.map fst

    // Reconstruct the path
    let rec reconstruct (acc: (PathNumber * Index) list) (current: PathNumber * Index) : ((PathNumber * Index) list option) =
        match Map.containsKey current previous, List.exists ((=) current) acc with
        | true, false -> reconstruct (current :: acc) (previous.[current]) 
        | false, _ when List.contains current startPoints -> Some (current :: acc)
        | _ -> None
    
    Seq.choose (reconstruct []) goals |> Seq.minBy (Seq.length)


let shortestPath = findPath {x=2; y=2} paths pathIndex |> List.map (fun (num, ix) -> paths.[num].[ix])


In [1]:
#!fsharp
shortestPath

index,type,Item
0,FSI_0471+PathElement+PathStart,{ x = 2 y = 2 }
1,FSI_0471+PathElement+Open,{ x = 3 y = 2 }
2,FSI_0471+PathElement+Open,{ x = 4 y = 2 }
3,FSI_0471+PathElement+Open,{ x = 5 y = 2 }
4,FSI_0471+PathElement+Open,{ x = 6 y = 2 }
5,FSI_0471+PathElement+Open,{ x = 6 y = 3 }
6,FSI_0471+PathElement+Open,{ x = 5 y = 3 }
7,FSI_0471+PathElement+Open,{ x = 5 y = 4 }
8,FSI_0471+PathElement+Open,{ x = 5 y = 5 }
9,FSI_0471+PathElement+Open,{ x = 5 y = 6 }


In [1]:
#!fsharp
open XPlot.Plotly

let deconstructCoordinateList l =
    List.fold (fun (xs, ys) c -> c.x :: xs, c.y :: ys) ([], []) l

let coordinateListToScatter l =
    let xs, ys = deconstructCoordinateList l
    Scatter(x = xs, y = ys, mode = "lines", line = Line(color = "grey", width = 0.33))

let stylizedCoordinateListToScatter l =
    let xs, ys = deconstructCoordinateList l
    Scatter(x = xs, y = ys, mode = "lines")

let pathToScatter (p: Path) =
    p |> List.ofArray |> List.map coordinateOf |> coordinateListToScatter

(shortestPath |> List.map coordinateOf |> stylizedCoordinateListToScatter) :: (Array.map pathToScatter paths |> List.ofArray) 
|> Chart.Plot 
|> Chart.WithLayout (Layout(yaxis = Yaxis (autorange = "reversed")))
|> Chart.WithHeight 1000
|> Chart.WithWidth 1000

In [1]:
#!fsharp
let getRelativePosition a b =
        match a, b with
        | a, b when b.x = a.x + 1 -> Right
        | a, b when b.x = a.x - 1 -> Left
        | a, b when b.y = a.y - 1 -> PathInstruction.Up
        | a, b when b.y = a.y + 1 -> Down
        | _ -> failwith "Can't get relative position"



let rec pathToInstructions (acc: (Coordinate * PathInstruction) list) (coords: List<Coordinate>) =
    match acc, coords with
    | [], a :: [] -> failwith "Need more cordinates"
    | [], a :: b :: rest -> pathToInstructions [b, getRelativePosition a b] rest
    | (x, _) :: xs, y :: ys -> pathToInstructions ((y, getRelativePosition x y) :: acc) ys 
    | _, [] -> List.rev acc


let shortestPathInstruction = shortestPath |> List.map coordinateOf |> pathToInstructions [] |> List.map snd

let instructionToLetter (ins: PathInstruction) =
    match ins with
    | Right -> 'R'
    | Left -> 'L'
    | Down -> 'D'
    | Up -> 'U'
    | Finish -> 'F'
    | Start -> 'S'
    | Wall -> 'X'

let output = String.Join (",", (List.map instructionToLetter shortestPathInstruction))

System.IO.File.WriteAllText ("./android_output.txt", output)
